### Variables

In [ ]:
host = ''
port = "5432"
dbname = ""
user = ""
password = ""
schema = ""
output_path = "/Volumes/path/to/avro_backup/"
jdbc_url = f"jdbc:postgresql://{host}:{port}/{dbname}"
driver = "org.postgresql.Driver"

jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

In [ ]:
import psycopg2
import os

def connect_db(host, port, dbname, user, password):
    try:
        connection = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        return connection
    except Exception as e:
        print(f"Error conectando a la base de datos: {e}")
        return None
    

def get_tables(conn, schema_name):
    cursor = conn.cursor()
    query = f"""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = '{schema}' AND table_type = 'BASE TABLE';
    """
    cursor.execute(query)
    tables = [row[0] for row in cursor.fetchall()]
    cursor.close()
    conn.close()
    return tables

def backup_table_to_avro(table_name, schema_name):
    table_full_name = f"{schema_name}.{table_name}"
    
    df = spark.read.jdbc(url=jdbc_url, table=table_full_name, properties=jdbc_properties)
    
    avro_output = os.path.join(output_path, f"{table_name}.avro")
    df.write.format("avro").mode("overwrite").save(avro_output)
    print(f"Backup de {table_name} completado y almacenado en {avro_output}")





In [ ]:
connection = connect_db(host, port, dbname, user, password)

if connection:
    
    tables = get_tables(connection, schema)

    for table in tables:
        backup_table_to_avro(table, schema)